In [1]:
#from shutil import copy
#from shutil import copyfile
import os
import sys
import cv2 as cv2
import glob
#import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from os import walk
import IPython
#IPython.embed() # to debug in notebook
import random
print("Successfully imported all")

Successfully imported all


In [2]:
# Credit: https://stackoverflow.com/questions/287871/print-in-terminal-with-colors
class CMD_C:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'   # End formatting
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    

# INPUT ARGS

In [3]:
print(sys.argv)
if sys.argv[0] != "Combine_Image_Dir_To_Video.py":    # Running in jupter notebook, or python file named differently
    # Use default arguments
    input_img_dir = "../LaneNet/output_frames" 
    save_as_name = "COMBINED_VIDEO.avi"
else:
    input_img_dir = sys.argv[-2]
    save_as_name = sys.argv[-1]
    
print("Source image directory: ", input_img_dir)

if os.path.exists(input_img_dir) == False:
    print(CMD_C.FAIL, input_img_dir, CMD_C.ENDC, " DOESN'T EXISTS", sep="")
    assert False    # Because jupyter notebook doesn't like quit()


print("Saving combined video as: ", save_as_name)
cwd = os.getcwd()
print("CWD: ", cwd)

save_as_name = cwd + "/" + save_as_name
print("Path to save combined video: ", save_as_name)

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-772a09d9-3f90-4493-883c-1e1f45e7852b.json']
Source image directory:  ../LaneNet/output_frames
Saving combined video as:  COMBINED_VIDEO.avi
CWD:  /root/atlas_root/home/cjcramer/lane_lines/Scripts
Path to save combined video:  /root/atlas_root/home/cjcramer/lane_lines/Scripts/COMBINED_VIDEO.avi


# GATHER PICTURES

In [4]:
def acceptable_file(file_path):
    ext = file_path.split("/")[-1].split(".")[-1]
    ext = ext.lower()
    accepted = ["png", "jpg", "jpeg"]
    return ext in accepted

#picture_paths = glob.glob(input_img_dir + "/*")     # Collect all files
picture_paths = []
for root, dirs, files in os.walk(input_img_dir):
    for file in files:
         picture_paths.append(os.path.join(root, file))
                
print("File in picture path: ", len(picture_paths))
picture_paths_passed = [p for p in picture_paths if acceptable_file(p)]
num_files_removed = len(picture_paths) - len(picture_paths_passed)
if num_files_removed > 0:
    print(CMD_C.FAIL, num_files_removed, CMD_C.ENDC, " FILES NOT INCLUDED SINCE THEY AREN'T png, jpg, or jpeg", sep="")
picture_paths = picture_paths_passed
if len(picture_paths) == 0:
    print("There are images, ending")
    assert False

# Make sure images are in order?
#for i in range(10):
#    print(picture_paths[i])
    
    
# If anyone else besides Calvin is running this they probably need to change this in order to get the frames in the correct order
def order_files(path):
    return 10000 * int(path.split("/")[-2].split("g")[-1]) + int(path.split("/")[-1].split(".")[0])
picture_paths = sorted(picture_paths, key=order_files)

print()
for i in range(len(picture_paths)):
    print(picture_paths[i])

File in picture path:  1260

../LaneNet/output_frames/vid_seg0/1.jpg
../LaneNet/output_frames/vid_seg0/2.jpg
../LaneNet/output_frames/vid_seg0/3.jpg
../LaneNet/output_frames/vid_seg0/4.jpg
../LaneNet/output_frames/vid_seg0/5.jpg
../LaneNet/output_frames/vid_seg0/6.jpg
../LaneNet/output_frames/vid_seg0/7.jpg
../LaneNet/output_frames/vid_seg0/8.jpg
../LaneNet/output_frames/vid_seg0/9.jpg
../LaneNet/output_frames/vid_seg0/10.jpg
../LaneNet/output_frames/vid_seg0/11.jpg
../LaneNet/output_frames/vid_seg0/12.jpg
../LaneNet/output_frames/vid_seg0/13.jpg
../LaneNet/output_frames/vid_seg0/14.jpg
../LaneNet/output_frames/vid_seg0/15.jpg
../LaneNet/output_frames/vid_seg0/16.jpg
../LaneNet/output_frames/vid_seg0/17.jpg
../LaneNet/output_frames/vid_seg0/18.jpg
../LaneNet/output_frames/vid_seg0/19.jpg
../LaneNet/output_frames/vid_seg0/20.jpg
../LaneNet/output_frames/vid_seg1/1.jpg
../LaneNet/output_frames/vid_seg1/2.jpg
../LaneNet/output_frames/vid_seg1/3.jpg
../LaneNet/output_frames/vid_seg1/4.jpg


../LaneNet/output_frames/vid_seg19/14.jpg
../LaneNet/output_frames/vid_seg19/15.jpg
../LaneNet/output_frames/vid_seg19/16.jpg
../LaneNet/output_frames/vid_seg19/17.jpg
../LaneNet/output_frames/vid_seg19/18.jpg
../LaneNet/output_frames/vid_seg19/19.jpg
../LaneNet/output_frames/vid_seg19/20.jpg
../LaneNet/output_frames/vid_seg20/1.jpg
../LaneNet/output_frames/vid_seg20/2.jpg
../LaneNet/output_frames/vid_seg20/3.jpg
../LaneNet/output_frames/vid_seg20/4.jpg
../LaneNet/output_frames/vid_seg20/5.jpg
../LaneNet/output_frames/vid_seg20/6.jpg
../LaneNet/output_frames/vid_seg20/7.jpg
../LaneNet/output_frames/vid_seg20/8.jpg
../LaneNet/output_frames/vid_seg20/9.jpg
../LaneNet/output_frames/vid_seg20/10.jpg
../LaneNet/output_frames/vid_seg20/11.jpg
../LaneNet/output_frames/vid_seg20/12.jpg
../LaneNet/output_frames/vid_seg20/13.jpg
../LaneNet/output_frames/vid_seg20/14.jpg
../LaneNet/output_frames/vid_seg20/15.jpg
../LaneNet/output_frames/vid_seg20/16.jpg
../LaneNet/output_frames/vid_seg20/17.jpg
.

# COMBINE PICTURES

In [5]:
first_img = cv2.imread(picture_paths[-1])
height, width, layers =  first_img.shape
print("Height, width, layers of each image: ", height, width, layers)

# Hey you, Calvin, try 40, 50, 60 fps.
video_writer = cv2.VideoWriter(save_as_name, cv2.VideoWriter_fourcc(*"MJPG"), 50, (width, height))
print("Video writer: ", video_writer)

for i, pp in enumerate(picture_paths):
    img = cv2.imread(pp)
    #plt.imshow(img)
    video_writer.write(img)
    print("Wrote frame ", CMD_C.OKGREEN, str(i+1), CMD_C.ENDC, " / ", str(len(picture_paths)))

video_writer.release()
cv2.destroyAllWindows()

Height, width, layers of each image:  720 1280 3
Video writer:  <VideoWriter 0x7f996a6cbed0>
Wrote frame   1   /  1260
Wrote frame   2   /  1260
Wrote frame   3   /  1260
Wrote frame   4   /  1260
Wrote frame   5   /  1260
Wrote frame   6   /  1260
Wrote frame   7   /  1260
Wrote frame   8   /  1260
Wrote frame   9   /  1260
Wrote frame   10   /  1260
Wrote frame   11   /  1260
Wrote frame   12   /  1260
Wrote frame   13   /  1260
Wrote frame   14   /  1260
Wrote frame   15   /  1260
Wrote frame   16   /  1260
Wrote frame   17   /  1260
Wrote frame   18   /  1260
Wrote frame   19   /  1260
Wrote frame   20   /  1260
Wrote frame   21   /  1260
Wrote frame   22   /  1260
Wrote frame   23   /  1260
Wrote frame   24   /  1260
Wrote frame   25   /  1260
Wrote frame   26   /  1260
Wrote frame   27   /  1260
Wrote frame   28   /  1260
Wrote frame   29   /  1260
Wrote frame   30   /  1260
Wrote frame   31   /  1260
Wrote frame   32   /  1260
Wrote frame   33   /  1260
Wrote frame   34   /  126

Wrote frame   230   /  1260
Wrote frame   231   /  1260
Wrote frame   232   /  1260
Wrote frame   233   /  1260
Wrote frame   234   /  1260
Wrote frame   235   /  1260
Wrote frame   236   /  1260
Wrote frame   237   /  1260
Wrote frame   238   /  1260
Wrote frame   239   /  1260
Wrote frame   240   /  1260
Wrote frame   241   /  1260
Wrote frame   242   /  1260
Wrote frame   243   /  1260
Wrote frame   244   /  1260
Wrote frame   245   /  1260
Wrote frame   246   /  1260
Wrote frame   247   /  1260
Wrote frame   248   /  1260
Wrote frame   249   /  1260
Wrote frame   250   /  1260
Wrote frame   251   /  1260
Wrote frame   252   /  1260
Wrote frame   253   /  1260
Wrote frame   254   /  1260
Wrote frame   255   /  1260
Wrote frame   256   /  1260
Wrote frame   257   /  1260
Wrote frame   258   /  1260
Wrote frame   259   /  1260
Wrote frame   260   /  1260
Wrote frame   261   /  1260
Wrote frame   262   /  1260
Wrote frame   263   /  1260
Wrote frame   264   /  1260
Wrote frame   265   

Wrote frame   459   /  1260
Wrote frame   460   /  1260
Wrote frame   461   /  1260
Wrote frame   462   /  1260
Wrote frame   463   /  1260
Wrote frame   464   /  1260
Wrote frame   465   /  1260
Wrote frame   466   /  1260
Wrote frame   467   /  1260
Wrote frame   468   /  1260
Wrote frame   469   /  1260
Wrote frame   470   /  1260
Wrote frame   471   /  1260
Wrote frame   472   /  1260
Wrote frame   473   /  1260
Wrote frame   474   /  1260
Wrote frame   475   /  1260
Wrote frame   476   /  1260
Wrote frame   477   /  1260
Wrote frame   478   /  1260
Wrote frame   479   /  1260
Wrote frame   480   /  1260
Wrote frame   481   /  1260
Wrote frame   482   /  1260
Wrote frame   483   /  1260
Wrote frame   484   /  1260
Wrote frame   485   /  1260
Wrote frame   486   /  1260
Wrote frame   487   /  1260
Wrote frame   488   /  1260
Wrote frame   489   /  1260
Wrote frame   490   /  1260
Wrote frame   491   /  1260
Wrote frame   492   /  1260
Wrote frame   493   /  1260
Wrote frame   494   

Wrote frame   688   /  1260
Wrote frame   689   /  1260
Wrote frame   690   /  1260
Wrote frame   691   /  1260
Wrote frame   692   /  1260
Wrote frame   693   /  1260
Wrote frame   694   /  1260
Wrote frame   695   /  1260
Wrote frame   696   /  1260
Wrote frame   697   /  1260
Wrote frame   698   /  1260
Wrote frame   699   /  1260
Wrote frame   700   /  1260
Wrote frame   701   /  1260
Wrote frame   702   /  1260
Wrote frame   703   /  1260
Wrote frame   704   /  1260
Wrote frame   705   /  1260
Wrote frame   706   /  1260
Wrote frame   707   /  1260
Wrote frame   708   /  1260
Wrote frame   709   /  1260
Wrote frame   710   /  1260
Wrote frame   711   /  1260
Wrote frame   712   /  1260
Wrote frame   713   /  1260
Wrote frame   714   /  1260
Wrote frame   715   /  1260
Wrote frame   716   /  1260
Wrote frame   717   /  1260
Wrote frame   718   /  1260
Wrote frame   719   /  1260
Wrote frame   720   /  1260
Wrote frame   721   /  1260
Wrote frame   722   /  1260
Wrote frame   723   

Wrote frame   919   /  1260
Wrote frame   920   /  1260
Wrote frame   921   /  1260
Wrote frame   922   /  1260
Wrote frame   923   /  1260
Wrote frame   924   /  1260
Wrote frame   925   /  1260
Wrote frame   926   /  1260
Wrote frame   927   /  1260
Wrote frame   928   /  1260
Wrote frame   929   /  1260
Wrote frame   930   /  1260
Wrote frame   931   /  1260
Wrote frame   932   /  1260
Wrote frame   933   /  1260
Wrote frame   934   /  1260
Wrote frame   935   /  1260
Wrote frame   936   /  1260
Wrote frame   937   /  1260
Wrote frame   938   /  1260
Wrote frame   939   /  1260
Wrote frame   940   /  1260
Wrote frame   941   /  1260
Wrote frame   942   /  1260
Wrote frame   943   /  1260
Wrote frame   944   /  1260
Wrote frame   945   /  1260
Wrote frame   946   /  1260
Wrote frame   947   /  1260
Wrote frame   948   /  1260
Wrote frame   949   /  1260
Wrote frame   950   /  1260
Wrote frame   951   /  1260
Wrote frame   952   /  1260
Wrote frame   953   /  1260
Wrote frame   954   

Wrote frame   1143   /  1260
Wrote frame   1144   /  1260
Wrote frame   1145   /  1260
Wrote frame   1146   /  1260
Wrote frame   1147   /  1260
Wrote frame   1148   /  1260
Wrote frame   1149   /  1260
Wrote frame   1150   /  1260
Wrote frame   1151   /  1260
Wrote frame   1152   /  1260
Wrote frame   1153   /  1260
Wrote frame   1154   /  1260
Wrote frame   1155   /  1260
Wrote frame   1156   /  1260
Wrote frame   1157   /  1260
Wrote frame   1158   /  1260
Wrote frame   1159   /  1260
Wrote frame   1160   /  1260
Wrote frame   1161   /  1260
Wrote frame   1162   /  1260
Wrote frame   1163   /  1260
Wrote frame   1164   /  1260
Wrote frame   1165   /  1260
Wrote frame   1166   /  1260
Wrote frame   1167   /  1260
Wrote frame   1168   /  1260
Wrote frame   1169   /  1260
Wrote frame   1170   /  1260
Wrote frame   1171   /  1260
Wrote frame   1172   /  1260
Wrote frame   1173   /  1260
Wrote frame   1174   /  1260
Wrote frame   1175   /  1260
Wrote frame   1176   /  1260
Wrote frame   

In [6]:
print("All done")

All done
